# Object Detection Tutorial with Faster R-CNN Transfer Learning
## Geopython 2019, Adrian F. Meyer
Parts of this tutorial are based on the [Medium article](https://hackernoon.com/object-detection-in-google-colab-with-custom-dataset-5a7bb2b0e97e) by RomRoc, 2018.

**In this Tutorial we will learn, how to use the Tensorflow Object Detection library, to detect solar panels on tiles of an aerial orthomosaic.**

![alt text](https://wilddrone.ch/wp-content/uploads/2019/06/solardetector.jpg)

The code, libraries and cloud environments used in this tutorial are currently available for free and are generally released open source.
You will need a Google account to execute the Notebook in its entirety, because it is meant to be executed on the Google Colab platform.

*Go to [Google Colab](https://colab.research.google.com/) and upload the notebook there. Make sure that you use Python 3 and GPU hardware acceleration as Runtime Environment.*

The dataset provided is based on the publically available SwissImage orthomosaic by [SwissTopo](https://map.geo.admin.ch/?topic=swisstopo&lang=de&bgLayer=ch.swisstopo.swissimage). 
The images and annotations can be downloaded as Zip File (31 Mbyte) here:
[https://drive.google.com/file/d/1i9RlEJTeB-KRauwhuMp-Vl13-Z3Yrj_y/view?usp=sharing](https://drive.google.com/file/d/1i9RlEJTeB-KRauwhuMp-Vl13-Z3Yrj_y/view?usp=sharing)



![alt text](https://wilddrone.ch/wp-content/uploads/2019/06/Tiles.jpg)

# Download Tensorflow Repo and Python Modules
By executing the first code snippet you initialize your virtual linux-style machine. Use The little arrow ">" in the top left corner to view the file system of your hosted system.
You can use UNIX-style terminal commands by using the prefix % and elevated priviledge commands for installations with the prefix !.

In [14]:
#make sure numpy is downgraded for compatibility reasons. 
#It will throw an error, which is not a problem, just click on the newly
#appeared Button "Restart Runtime"
!pip install numpy==1.17.4

     |████████████████████████████████| 20.0 MB 7.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.17.4 which is incompatible.
kapre 0.3.5 requires numpy>=1.18.5, but you have numpy 1.17.4 which is incompatible.
jaxlib 0.1.71+cuda111 requires numpy>=1.18, but you have numpy 1.17.4 which is incompatible.
jax 0.2.21 requires numpy>=1.18, but you have numpy 1.17.4 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
%cd /content
%tensorflow_version 1.x

#make sure to be in /root and that tensorflow is running in version 1.15.2
import tensorflow as tf
print(tf.__version__)

/content
1.15.2


In [3]:
"""
This allows you to check which GPU you have been allocated. Google offers free
Tesla T4, Tesla K80, Tesla P100 (the P100 hax 1.6x more GFLOPs and 3x the memory bandwith than K80, the T4 is fairly slow).
In theory you can restart the environment until you have the fast one. 
For testing and learning it doesn't really matter.
"""
  
#We have to work with Tensorflow 1.15.2 for code compatibility reasons; by now TF v2 is available.
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
!nvidia-smi

Sun Oct 10 13:20:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    58W / 149W |     60MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
%cd /content
  
!git clone https://github.com/tensorflow/models.git

/content
Cloning into 'models'...
remote: Enumerating objects: 64364, done.
remote: Total 64364 (delta 0), reused 0 (delta 0), pack-reused 64364
Receiving objects: 100% (64364/64364), 575.04 MiB | 29.62 MiB/s, done.
Resolving deltas: 100% (45043/45043), done.


In [5]:
%cd /content
  
"""
This repository contains a number of different models implemented in TensorFlow:
The official models are a collection of example models that use TensorFlow's high-level APIs. They are intended to be well-maintained, tested, and kept up to date with the latest stable TensorFlow API. They should also be reasonably optimized for fast performance while still being easy to read. We especially recommend newer TensorFlow users to start here.
The research models are a large collection of models implemented in TensorFlow by researchers. They are not officially supported or available in release branches; it is up to the individual researchers to maintain the models and/or provide support on issues and pull requests.
The samples folder contains code snippets and smaller models that demonstrate features of TensorFlow, including code presented in various blog posts.
"""  

!apt-get install protobuf-compiler python-tk

"""
Protocol buffers are Google's language-neutral, platform-neutral, extensible mechanism for serializing structured data; similar to JSON or XML.
"""

!pip install Cython contextlib2 pillow lxml matplotlib PyDrive
"""
These context modules are necessary python pachages. Especially Cython is important: It allows to call native C or C++ bindings from within python.
"""

!pip install pycocotools
"""
COCO is a large image dataset designed for object detection, segmentation, person keypoints detection, stuff segmentation, and caption generation. 
"""

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=. #This initializes/compiles the Tensorflow Protobuf evnironment.

import os
os.environ['PYTHONPATH'] += ':/models/research/:/models/research/slim/'


/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
/content/models/research


# Install Tensorflow on Virtual Machine

In [10]:
%cd /content/models/research/slim
!python setup.py build
!python setup.py install > /dev/null
"""
This snippet builds and installs the Tensorflow API from the cloned git source.
"""

/content/models/research/slim
running build
running build_py
creating build
creating build/lib
creating build/lib/preprocessing
copying preprocessing/preprocessing_factory.py -> build/lib/preprocessing
copying preprocessing/__init__.py -> build/lib/preprocessing
copying preprocessing/lenet_preprocessing.py -> build/lib/preprocessing
copying preprocessing/inception_preprocessing.py -> build/lib/preprocessing
copying preprocessing/vgg_preprocessing.py -> build/lib/preprocessing
copying preprocessing/cifarnet_preprocessing.py -> build/lib/preprocessing
creating build/lib/nets
copying nets/i3d_test.py -> build/lib/nets
copying nets/inception_v2_test.py -> build/lib/nets
copying nets/s3dg.py -> build/lib/nets
copying nets/i3d.py -> build/lib/nets
copying nets/alexnet_test.py -> build/lib/nets
copying nets/i3d_utils.py -> build/lib/nets
copying nets/dcgan.py -> build/lib/nets
copying nets/inception_v1.py -> build/lib/nets
copying nets/__init__.py -> build/lib/nets
copying nets/mobilenet_v1_e

'\nThis snippet builds and installs the Tensorflow API from the cloned git source.\n'

In [11]:
%cd slim
!pip install -e .

%cd ..
!python object_detection/builders/model_builder_test.py
"""
This tests if the installation was successful. The Tests should yield the output [ RUN  OK ]
"""

[Errno 2] No such file or directory: 'slim'
/content/models/research/slim
Obtaining file:///content/models/research/slim
  Attempting uninstall: slim
    Found existing installation: slim 0.1
    Uninstalling slim-0.1:
      Successfully uninstalled slim-0.1
  Running setup.py develop for slim
/content/models/research
Traceback (most recent call last):
  File "object_detection/builders/model_builder_test.py", line 21, in <module>
    from object_detection.builders import model_builder
ModuleNotFoundError: No module named 'object_detection'


'\nThis tests if the installation was successful. The Tests should yield the output [ RUN  OK ]\n'

#Upload and Import Dataset

In [1]:
%cd /content/drive/MyDrive
!wget http://2019.geopython.net/data/solar.zip
"""
We download the images, annotation files and independent test samples into the datalab folder.
"""

/datalab
--2021-10-10 10:21:10--  http://2019.geopython.net/data/solar.zip
Resolving 2019.geopython.net (2019.geopython.net)... 147.86.7.67
Connecting to 2019.geopython.net (2019.geopython.net)|147.86.7.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33438193 (32M) [application/zip]
Saving to: ‘solar.zip’

solar.zip           100%[===================>]  31.89M  74.8MB/s    in 0.4s    

2021-10-10 10:21:11 (74.8 MB/s) - ‘solar.zip’ saved [33438193/33438193]



'\nWe download the images, annotation files and independent test samples into the datalab folder.\n'

In [13]:
%cd /content/drive/MyDrive/
!unzip main.zip #Scroll through the unzip output to get an idea of the datalab folder content.

/content/drive/MyDrive
Archive:  main.zip
   creating: main/
   creating: main/annotations/
   creating: main/annotations/xmls/
  inflating: main/annotations/xmls/frame (1).xml  
  inflating: main/annotations/xmls/frame (10).xml  
  inflating: main/annotations/xmls/frame (11).xml  
  inflating: main/annotations/xmls/frame (12).xml  
  inflating: main/annotations/xmls/frame (13).xml  
  inflating: main/annotations/xmls/frame (14).xml  
  inflating: main/annotations/xmls/frame (15).xml  
  inflating: main/annotations/xmls/frame (16).xml  
  inflating: main/annotations/xmls/frame (2).xml  
  inflating: main/annotations/xmls/frame (3).xml  
  inflating: main/annotations/xmls/frame (4).xml  
  inflating: main/annotations/xmls/frame (5).xml  
  inflating: main/annotations/xmls/frame (6).xml  
  inflating: main/annotations/xmls/frame (7).xml  
  inflating: main/annotations/xmls/frame (8).xml  
  inflating: main/annotations/xmls/frame (9).xml  
  inflating: main/annotations/xmls/framee (1).xml

You can download one of the XML files to check the structure of the PASCAL VOC Annotation format.
Here is an example:



```
<annotation>
	<folder>sol2</folder>
	<filename>solar_10.JPG</filename>
	<path>C:\temp\sol2\solar_10.JPG</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>901</width>
		<height>791</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>solar</name>
		<pose>Unspecified</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>637</xmin>
			<ymin>152</ymin>
			<xmax>901</xmax>
			<ymax>591</ymax>
		</bndbox>
	</object>
</annotation>

```



We need to write our label name (in this case "solar") into a config file defining all detectable classes.
In our case it is just on class.

Then we iterate through all image files to extract the file names (paths are not relevant) we want to use for training and validation.

In [12]:
%cd ..
%cd /content/drive/MyDrive/main



/content/models
/content/drive/MyDrive/main


In [13]:
!echo "item { id: 1 name: 'True'}" > label_map.pbtxt

image_files=os.listdir('images')
im_files=[x.split('.')[0] for x in image_files]
with open('annotations/trainval.txt', 'w') as text_file:
  for row in im_files:
    text_file.write(row + '\n')

#Data and Model Preparation
## Generate Bounding Boxes on Images for RPN Network Training
The same process need to be performed with the XML Annotation files.
Additionally, we convert the images to PNG format for easier access.

In [14]:
%cd /content/drive/MyDrive/main/annotations

!mkdir trimaps

from PIL import Image
image = Image.new('RGB', (901, 791))

for filename in os.listdir('xmls'):
  filename = os.path.splitext(filename)[0]
  image.save('trimaps/' + filename + '.png')

/content/drive/MyDrive/main/annotations


##Generate Labelled Tensor Matrices (tf_records)
The Tensorflow Record files contain the actual input data for the Machine Learning process in binary format.
An API specific script can do the job for us.
We use the famous "pet faces model" in our transfer learning process.
We need to split the dataset at this point into training and validation data.
70% of our data (148 of 212 images) will be used for training, the remaining 30% for validation (64 images)

In [37]:
#%cd /content/drive/MyDrive/main\
%cd /content/models/research/slim

!python /content/models/research/object_detection/dataset_tools/create_pet_tf_record.py --label_map_path=label_map.pbtxt --data_dir=. --output_dir=. --num_shards=1

!mv pet_faces_train.record-00000-of-00001 tf_train.record

!mv pet_faces_val.record-00000-of-00001 tf_val.record

/content/models/research/slim
Traceback (most recent call last):
  File "/content/models/research/object_detection/dataset_tools/create_pet_tf_record.py", line 42, in <module>
    from object_detection.dataset_tools import tf_record_creation_util
ModuleNotFoundError: No module named 'object_detection'
mv: cannot stat 'pet_faces_train.record-00000-of-00001': No such file or directory
mv: cannot stat 'pet_faces_val.record-00000-of-00001': No such file or directory


##Download the Model Checkpoint you want to use for Transfer Learning
Many different COCO pretrained neural models can be used for bounding box related object detection with Tensorflow.
They all have different advantages or disadvantages (e.g. inferencing speed, accuracy, easy to train, etc.).

An overview can be found with the [TF Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).



In [18]:
%cd /content/drive/MyDrive/main
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
  
%cd /content/drive/MyDrive/main
!tar -xvzf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

%cd /content/drive/MyDrive/main
!mv faster_rcnn_inception_v2_coco_2018_01_28 pretrained_model

/content/drive/MyDrive/main
--2021-10-10 13:29:15--  http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.125.128, 2607:f8b0:4001:c16::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.125.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149119618 (142M) [application/x-tar]
Saving to: ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’

faster_rcnn_incepti 100%[===================>] 142.21M  45.5MB/s    in 3.1s    

2021-10-10 13:29:18 (45.5 MB/s) - ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’ saved [149119618/149119618]

/content/drive/MyDrive/main
faster_rcnn_inception_v2_coco_2018_01_28/
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
faster_rcnn_inception_v2_coco_2018_01_28/checkpoint
faster_rcnn_inception_v2_coco_2018_01_28/pipeline.config
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-0

##Configure the Paths and Training Parameters
This specifies which files and model checkpoints should be used for the trainings process.

In [21]:
#%cd \content\

import re

#filename = '/datalab/pretrained_model/pipeline.config'
filename = '/content/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config'
with open(filename) as f:
  s = f.read()
with open(filename, 'w') as f:
  s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', '/content/drive/MyDrive/main/pretrained_model/model.ckpt', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/pet_faces_train.record-\?\?\?\?\?-of-00010', '/content/drive/MyDrive/main/tf_train.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/pet_faces_val.record-\?\?\?\?\?-of-00010', '/content/drive/MyDrive/main/tf_test.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/pet_label_map.pbtxt', '/content/drive/MyDrive/main/label_map.pbtxt', s)
  f.write(s)

# Training on GPU
The execution of this snippet might take a while. Normally the training of 3000 steps should take about 10 minutes (approx. 12 seconds for 100 steps). 

As a rough estimate, the loss value of Faster RCNN models should fall below 0.05 over a few thousand steps and then the training can be aborted. 

We configure automatic termination after 3'000 Steps, in productive trainings as much as 100'000-200'000 Steps can be neccesary.

![alt text](https://cdn-images-1.medium.com/max/800/1*qGb5XNny5G8PrGQ5sejFvw.png)

This graph shows the expected training progess of the model with the supervision tool "Tensorboard".

In [33]:
%cd /content/models/research/object_detection
!pip install object_detection

/content/models/research
ERROR: Could not find a version that satisfies the requirement object_detection (from versions: none)
ERROR: No matching distribution found for object_detection


In [28]:
#%cd /content/drive/MyDrive/main

#%cp -R /content

!python /content/models/research/object_detection/model_main.py\
    --pipeline_config_path=/content/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config \
    --model_dir=/content/drive/MyDrive/main/trained \
    --train_dir=/content/drive/MyDrive/main/trained \
    --logtostderr \
    --logdir=/content/drive/MyDrive/main/trained \
    --num_train_steps=3000 \
    --num_eval_steps=500 \
    --max_evals=0 

Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main.py", line 25, in <module>
    from object_detection import model_lib
ModuleNotFoundError: No module named 'object_detection'


# Export Inference Graph
Inferencing means to apply the model to images which haven't been used for training.

We reserved a few images to check if our model performs correctly.

The frozen Inference Graph gets generated from the last model checkpoint and contains all elements of the model neccesary to perform inference (also on weaker hardware), but it cannot be used to continue training the model.

In [ ]:
%cd /datalab

lst = os.listdir('trained')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

!python ~/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/root/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config \
    --output_directory=fine_tuned_model \
    --trained_checkpoint_prefix=trained/$last_model

##Alternative: Import a Finished Inference Graph
Uncomment this section, if you want to download a finished inference graph if you could not finish the training.

In [ ]:
#Alternative to training, you can download a fully trained model inference graph by uncommenting (#) the following lines

#%cd /datalab
#!wget http://2019.geopython.net/data/trained.tar.gz
#%cd /datalab
#!tar -xzf trained.tar.gz

# Run Inference

In [ ]:
%cd /root/models/research/object_detection




import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

#if tf.__version__ < '1.4.0':
#  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
  

  
  
# This is needed to display the images.
%matplotlib inline




from utils import label_map_util

from utils import visualization_utils as vis_util




# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/datalab/fine_tuned_model' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/datalab', 'label_map.pbtxt')

NUM_CLASSES = 37




detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
    
    
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)




def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)




# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/datalab/testsamples/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.JPG'.format(i)) for i in range(1, 4) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)




def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict




for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

## Run Inference on Additional Samples

In [ ]:
%cd /root/models/research/object_detection




import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

#if tf.__version__ < '1.4.0':
#  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
  

  
  
# This is needed to display the images.
%matplotlib inline




from utils import label_map_util

from utils import visualization_utils as vis_util




# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/datalab/fine_tuned_model' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/datalab', 'label_map.pbtxt')

NUM_CLASSES = 37




detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
    
    
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)




def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)




# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/datalab/images/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'solar_{}.JPG'.format(i)) for i in range(150, 166) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)




def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict




for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

In [25]:
import pandas as pd
import numpy as np
licensees = pd.read_csv("/content/sample_data/train_labels.csv", index_col=0)[1:]
licensees.head(10)

,xmin,xmax,ymin,ymax,filename,width,height
class,,,,,,,
Defect,0,115,3351,734,framee (2).jpg,3840,2160
Defect,0,115,3346,725,framee (3).jpg,3840,2160
Defect,0,129,3342,752,framee (4).jpg,3840,2160
Defect,0,115,3351,765,framee (5).jpg,3840,2160
Defect,5,115,3351,761,framee (6).jpg,3840,2160
Defect,0,134,3342,743,framee (7).jpg,3840,2160
Defect,5,134,3360,729,framee (8).jpg,3840,2160
Defect,0,134,3355,761,framee (9).jpg,3840,2160
Defect,5,79,3346,788,framee (1).jpg,3840,2160


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats # some useful stuff
wine_data = pd.read_csv("/content/sample_data/train_labels.csv")

In [10]:
i = int(0)
while (i<60):
  {
    if ('TRUE' == wine_data['class[i]']):
      wine_data['class[i]'] = 1
    else:
      wine_data['class[i]'] = 0
 
  i++;
  }

SyntaxError: ignored

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:

points = wine_data['class']
mu = points.mean()
sigma = points.std(ddof=0)
print("mu: ", mu, ", sigma:", sigma)

TypeError: ignored